# Example with a linear difference system

From the update (dalgebra-0.0.20220428.1815) we can treat now difference rings too. Let's see how to work with them with a very simple linear system:

In [1]:
%display latex
from dalgebra import *

The system has 2 state variables $C(n)$ and $S(n)$ and the difference system can be written using the parameters $K_{se}, I_0, K_{cp}$ and $K_{rb}$. We need to create these variables and set the difference ring such that everything is a constant:

$$\begin{array}{rcl}
    I_0 & \mapsto & \texttt{I_0}\\
    K_{se} & \mapsto & \texttt{K_se}\\
    K_{cp} & \mapsto & \texttt{K_cp}\\
    K_{rb} & \mapsto & \texttt{K_rb}
\end{array}$$

In [2]:
R.<I_0, K_se, K_cp, K_rb, K_sc> = QQ[]
DR = DifferenceRing(R)
## We update the variables
I_0, K_se, K_cp, K_rb, K_sc = DR.gens()
## We check that all these are constants
print("All are constants -> ", all(el.difference() == el for el in DR.gens()))

DR

All are constants ->  True


Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In order to build the system, we need to create the two difference variables $C(n)$ and $S(n)$. In the code, these will be represented by $\texttt{C}$ and $\texttt{S}$ such that $\texttt{C[k]} = C(n+k)$ and $\texttt{S[k]} = S(n+k)$. We do that with the class `DifferencePolynomialRing`:

In [3]:
DPR.<C,S> = DifferencePolynomialRing(DR)
DPR

Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In [4]:
C, S

(C_*, S_*)

At this point we can create a Difference system with the appropiate equations:

In [5]:
system = DifferenceSystem([
    C[1] - K_se*I_0 * S[0] - (1 - K_cp - K_rb) * C[0] + K_se*S[0]*C[0],
    S[1] - (1 - K_se * I_0) * S[0] - K_rb * C[0] - K_sc * C[0] * S[0]
], variables = [C,S])
system

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

Let us consider now the output variables, that will be another system only focused on $S(n)$, so it will have coefficients using some shifts of $C(n)$:

In [6]:
system_on_S = system.change_variables([S])
system_on_S

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

And what Sonia checked by hand is that the system formed by $(f_1, \sigma f_1, f_2)$ is a nice system to delete $S(n)$:

In [7]:
extended_system = system_on_S.extend_by_difference([1,0])
print(f"Can we eliminate {extended_system.variables}? --> {extended_system.is_sp2()}")
extended_system

Can we eliminate (S_*,)? --> True


System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	K_se*C_1*S_1 + C_2 + (K_cp + K_rb - 1)*C_1 + (-I_0*K_se)*S_1 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

In [8]:
extended_system.diff_resultant(alg_res="macaulay")

(-K_se^2*K_rb + K_se*K_cp*K_sc + K_se*K_rb*K_sc - K_se*K_sc)*C_0^2*C_1 + K_se*K_sc*C_0*C_1^2 + (I_0*K_se^2*K_rb - I_0*K_se*K_cp*K_sc - I_0*K_se*K_rb*K_sc + I_0*K_se*K_sc)*C_0^2 + (-I_0*K_se^2*K_cp + I_0*K_se^2 - I_0*K_se*K_sc)*C_0*C_1 + (-I_0*K_se^2 + K_se)*C_1^2 + (-K_se)*C_0*C_2 + (I_0^2*K_se^2*K_cp - I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + I_0*K_se)*C_0 + (I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - 2*I_0*K_se)*C_1 + I_0*K_se*C_2

We can also try to compute this resultant from the original system (since `diff_resultant` extends the system):

In [9]:
system_on_S.diff_resultant(alg_res="macaulay", verbose=True)

We start by extending the system up to bound 10
Trying the extension (0, 0)
Trying the extension (1, 0)
Found the valid extension (1, 0)


(-K_se^2*K_rb + K_se*K_cp*K_sc + K_se*K_rb*K_sc - K_se*K_sc)*C_0^2*C_1 + K_se*K_sc*C_0*C_1^2 + (I_0*K_se^2*K_rb - I_0*K_se*K_cp*K_sc - I_0*K_se*K_rb*K_sc + I_0*K_se*K_sc)*C_0^2 + (-I_0*K_se^2*K_cp + I_0*K_se^2 - I_0*K_se*K_sc)*C_0*C_1 + (-I_0*K_se^2 + K_se)*C_1^2 + (-K_se)*C_0*C_2 + (I_0^2*K_se^2*K_cp - I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + I_0*K_se)*C_0 + (I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - 2*I_0*K_se)*C_1 + I_0*K_se*C_2

In [143]:
res_y_is_c = system_on_S.diff_resultant(alg_res="macaulay")
coefficients_y_is_c = [el.wrapped for el in res_y_is_c.coefficients()]

In [145]:
with open("/home/anton/results/coefficients_y=c.txt", "w") as f:
    for coeff in coefficients_y_is_c:
        f.write(str(coeff) + "\n")
with open("/home/anton/results/resultant_y=c.txt", "w") as f:
    f.write(str(res_y_is_c))

### Approach with output variables

In the previous example, we kind of cheated, since we took ans an output variable one of the variables that were already in the system. More generically, we will take a new function $Y(n)$ as an output and this will be the function we need to preserve throughout the elimination.

For example, consider te previous system with output variable $Y(n) = S(n) + C(n)$. This will be represented as follows:

In [10]:
DPR.<C,S,Y> = DifferencePolynomialRing(DR)
DPR

Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In [11]:
system = DifferenceSystem([
    C[1] - K_se*I_0 * S[0] - (1 - K_cp - K_rb) * C[0] + K_se*S[0]*C[0],
    S[1] - (1 - K_se * I_0) * S[0] - K_rb * C[0] - K_sc * C[0] * S[0],
    Y[0] - C[0] - S[0]
], variables = [C,S])
system

System over [Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
	-C_0 - S_0 + Y_0 == 0
}

We can try to apply the elimnation here right away:

In [12]:
#system.diff_resultant(alg_res="macaulay", verbose = True) ## too long

However, even though it founds a valid extension for the system in order to eliminate both $C(n)$ and $S(n)$ at the same time, the macaulay resultant we get is too elow to be computed. Let us see this valid system:

In [13]:
system.extend_by_operation([1,1,2]) # the extension was taken from the verbose output of the previous cell

System over [Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	K_se*C_1*S_1 + C_2 + (K_cp + K_rb - 1)*C_1 + (-I_0*K_se)*S_1 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
	(-K_sc)*C_1*S_1 + (-K_rb)*C_1 + S_2 + (I_0*K_se - 1)*S_1 == 0
	-C_0 - S_0 + Y_0 == 0
	-C_1 - S_1 + Y_1 == 0
	-C_2 - S_2 + Y_2 == 0
}

In [14]:
system.extend_by_operation([1,1,2]).is_sp2()

True

## An iterative approach

Using the structure of the system maybe we can eliminate one of the variables first, and then the other:

In [15]:
system

System over [Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
	-C_0 - S_0 + Y_0 == 0
}

In [16]:
eq_wo_C1 = system.parent()(str(system.subsystem([0,2], variables=[C]).diff_resultant(alg_res="macaulay"))); eq_wo_C1

(-K_se)*S_0^2 + K_se*S_0*Y_0 - S_1 + (-I_0*K_se - K_cp - K_rb + 1)*S_0 + Y_1 + (K_cp + K_rb - 1)*Y_0

In [17]:
eq_wo_C2 = system.parent()(str(system.subsystem([1,2], variables=[C]).diff_resultant(alg_res="macaulay"))); eq_wo_C2

K_sc*S_0^2 + (-K_sc)*S_0*Y_0 + S_1 + (I_0*K_se + K_rb - 1)*S_0 + (-K_rb)*Y_0

In [18]:
system_wo_C = DifferenceSystem([eq_wo_C1, eq_wo_C2], system.parent(), variables=[S]); system_wo_C

System over [Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	(-K_se)*S_0^2 + K_se*S_0*Y_0 - S_1 + (-I_0*K_se - K_cp - K_rb + 1)*S_0 + Y_1 + (K_cp + K_rb - 1)*Y_0 == 0
	K_sc*S_0^2 + (-K_sc)*S_0*Y_0 + S_1 + (I_0*K_se + K_rb - 1)*S_0 + (-K_rb)*Y_0 == 0
}

In [113]:
equations = list(system_wo_C.extend_by_operation([1,1]).equations())
variables = system_wo_C.extend_by_operation([1,1]).algebraic_variables()

In [114]:
def is_variable_in_equation(variable, equation):
    return equation.polynomial().degree(variable.polynomial()) > 0

def build_matrix_variables(equations, variables):
    M = []
    for equation in equations:
        row = []
        for variable in variables:
            poly = equation.polynomial().polynomial(variable.polynomial())
            ct = poly.constant_coefficient()
            row.append(poly-ct)
        M.append(row)
    return Matrix(variables[0].parent(), M)

def eliminate_variable_2eqs(variable, equation1, equation2):
    poly1 = equation1.polynomial().polynomial(variable.polynomial())
    poly2 = equation2.polynomial().polynomial(variable.polynomial())
    
    syl_matr = poly1.sylvester_matrix(poly2)
    det = syl_matr.determinant()
    if det == 0: raise ZeroDivisionError()
        
    return variable.parent()(det)

def eliminate_variable_gen(variable, equations):
    indices = [i for i in range(len(equations)) if is_variable_in_equation(variable, equations[i])]
    sorted(indices, key=lambda p : equations[0].polynomial().degree(variables[-1].polynomial()))
    new_eqs = [equations[i] for i in range(len(equations)) if (not i in indices)]
    for i in range(1, len(indices)):
        new_eqs.append(eliminate_variable_2eqs(variable, equations[indices[0]], equations[indices[i]]))
        
    return new_eqs

M = build_matrix_variables(equations, variables); M

[(-K_se)*S_0^2 + K_se*S_0*Y_0 + (-I_0*K_se - K_cp - K_rb + 1)*S_0                                                             -S_1                                                                0]
[                                                               0 (-K_se)*S_1^2 + K_se*S_1*Y_1 + (-I_0*K_se - K_cp - K_rb + 1)*S_1                                                             -S_2]
[        K_sc*S_0^2 + (-K_sc)*S_0*Y_0 + (I_0*K_se + K_rb - 1)*S_0                                                              S_1                                                                0]
[                                                               0         K_sc*S_1^2 + (-K_sc)*S_1*Y_1 + (I_0*K_se + K_rb - 1)*S_1                                                              S_2]

In [115]:
# eliminating all variables of S(n)
for var in variables[::-1]:
    equations = eliminate_variable_gen(var, equations)
    
print(f"Remaining equations: {len(equations)}")

Remaining equations: 1


In [116]:
equations[0].order(Y), equations[0].degree(), len(equations[0].monomials())

(2, 4, 21)

In [117]:
for coeff in equations[0].coefficients():
    print(f"(deg: {coeff.wrapped.degree()},\tn_mons: {len(coeff.wrapped.monomials())})")

(deg: 6,	n_mons: 3)
(deg: 8,	n_mons: 15)
(deg: 10,	n_mons: 47)
(deg: 11,	n_mons: 79)
(deg: 12,	n_mons: 84)
(deg: 5,	n_mons: 4)
(deg: 7,	n_mons: 22)
(deg: 9,	n_mons: 41)
(deg: 9,	n_mons: 54)
(deg: 11,	n_mons: 104)
(deg: 12,	n_mons: 159)
(deg: 13,	n_mons: 173)
(deg: 4,	n_mons: 5)
(deg: 8,	n_mons: 49)
(deg: 8,	n_mons: 70)
(deg: 12,	n_mons: 161)
(deg: 13,	n_mons: 214)
(deg: 14,	n_mons: 241)
(deg: 9,	n_mons: 31)
(deg: 11,	n_mons: 70)
(deg: 12,	n_mons: 90)


In [129]:
coefficients = [coeff.wrapped for coeff in equations[0].coefficients()]

In [131]:
g = gcd(coefficients)

In [132]:
coefficients = [el/g for el in coefficients]

In [133]:
print(coefficients[0])

K_se^3*K_sc^2 - K_se^2*K_sc^3


In [141]:
with open("/home/anton/results/coefficients_c+s.txt", "w") as f:
    for coeff in coefficients:
        f.write(str(coeff) + "\n")
with open("/home/anton/results/resultant_c+s.txt", "w") as f:
    f.write(str(equations[0]))